![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [50]:
import pandas as pd 
import numpy as np
import math
from datetime import datetime
from sklearn.ensemble import RandomForestClassifier
import joblib

from scipy.stats import mannwhitneyu
import scipy.stats as stats
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.utils.class_weight import compute_class_weight


In [51]:
def get_indicators(crypto_df, toggle_indicators):
    indicators = []

    #constants
    macd_short = 8
    macd_long = 12
    macd_signal = 6
    vwap_period = 16
    rsi_period = 16
    adx_period = 16
    cci_period = 24
    stoch_period = 12
    stoch_smooth = 6
    hma_period = 16
    cmo_period = 12
    uo_short = 6
    uo_mid = 12
    uo_long = 18

    #macd
    ema_short = crypto_df["close"].ewm(span=macd_short, adjust=False).mean()
    ema_long = crypto_df["close"].ewm(span=macd_long, adjust=False).mean()
    ema_signal = crypto_df["close"].ewm(span=macd_signal, adjust=False).mean()

    macd = ((ema_short - ema_long) - ema_signal) 
    if toggle_indicators["macd"]:
        indicators.append(macd)

    #vwap
    avg_price = (crypto_df["close"] + crypto_df["high"] + crypto_df["low"]) / 3
    weighted_price = (avg_price * crypto_df["volume"]).rolling(window=vwap_period).sum()
    volume = crypto_df["volume"].rolling(window=vwap_period).sum()
    vwap = weighted_price / volume
    #case when volume = 0 -> use prev vwap
    for i in range(len(vwap)):
        if np.isnan(vwap.iloc[i]) and i > 0:
            vwap.iloc[i] = vwap.iloc[i-1]
    vwap = (crypto_df["close"] - vwap) 
    if toggle_indicators["vwap"]:
        indicators.append(vwap)

    #rsi
    diffs = crypto_df["close"].diff()
    gain = (diffs.where(diffs > 0, 0)).rolling(window=rsi_period).mean()
    loss = (diffs.where(diffs < 0, 0)).rolling(window=rsi_period).mean()
    rs = gain / np.abs(loss)
    rsi = 100 - 100 / (1 + rs)
    #case when loss = 0 -> max momentum
    rsi = rsi.apply(lambda x: 100 if np.isnan(x) else x)
    if toggle_indicators["rsi"]:
        indicators.append(rsi)

    #adx
    high_low = crypto_df["high"] - crypto_df["low"]
    high_close = np.abs(crypto_df["high"] - crypto_df["close"].shift())
    low_close = np.abs(crypto_df["low"] - crypto_df["close"].shift())
    TR = high_low.combine(high_close, max).combine(low_close, max)

    plus_dm = crypto_df["high"].diff()
    minus_dm = crypto_df["low"].diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0

    smooth_TR = TR.rolling(window=adx_period).mean()
    smooth_plus_dm = plus_dm.rolling(window=adx_period).mean()
    smooth_minus_dm = abs(minus_dm.rolling(window=adx_period).mean())
    #Limiting denom
    smooth_TR = smooth_TR.apply(lambda x: 0.01 if abs(x) < 0.01 else x)

    plus_DI = (smooth_plus_dm / smooth_TR) * 100
    minus_DI = (smooth_minus_dm / smooth_TR) * 100
    adx_denom = plus_DI + minus_DI
    dx = ((plus_DI - minus_DI) / adx_denom) * 100
    #case when +DI + -DI = 0 -> no price movement
    dx = dx.apply(lambda x: 0 if np.isnan(x) else x)
    adx = dx.rolling(window=adx_period).mean()
    if toggle_indicators["adx"]:
        indicators.append(adx)

    #cci
    tp = (crypto_df["high"] + crypto_df["low"] + crypto_df["close"]) / 3

    sma_tp = tp.rolling(window=cci_period).mean()
    md = tp.rolling(window=cci_period).apply(lambda x: np.fabs(x - x.mean()).mean())

    # 0.015 is apparently a standard constant for 20-30% of cci values to indicate trend
    cci = (tp - sma_tp) / (0.015 * md)
    if toggle_indicators["cci"]:
        indicators.append(cci)
    
    #stochastic oscillator (stoch)
    stoch_low = crypto_df["low"].rolling(stoch_period).min()
    stoch_high = crypto_df["high"].rolling(stoch_period).max()
    stoch_k = 100 * (crypto_df["close"] - stoch_low) / (stoch_high - stoch_low)
    stoch_d = stoch_k.rolling(stoch_smooth).mean()
    stoch_signal = stoch_k - stoch_d
    if toggle_indicators["stoch"]:
        indicators.append(stoch_signal)
    
    #hma
    #traditionally uses custom weighted moving avg - used ema instead
    ema_half = crypto_df["close"].ewm(hma_period//2, adjust=False).mean()
    ema_full = crypto_df["close"].ewm(hma_period, adjust=False).mean()
    raw_hma = 2*ema_half - ema_full
    hma = crypto_df["close"] - (raw_hma.ewm(int(np.sqrt(hma_period)), adjust=False).mean()) 
    if toggle_indicators["hma"]:
        indicators.append(hma)

    #cmo
    high_closes = crypto_df["close"].diff().apply(lambda x: x if x > 0 else 0)
    low_closes = crypto_df["close"].diff().apply(lambda x: -x if x < 0 else 0)
    sum_high_closes = high_closes.rolling(cmo_period).sum()
    sum_low_closes = low_closes.rolling(cmo_period).sum()
    cmo = 100 * (sum_high_closes - sum_low_closes) / (sum_high_closes + sum_low_closes)
    if toggle_indicators["cmo"]:
        indicators.append(cmo)
    
    #uo
    bp = crypto_df["close"] - np.minimum(crypto_df["low"], crypto_df["close"].shift(1))
    tr_high = np.maximum(crypto_df['high'], crypto_df['close'].shift(1))
    tr_low = np.minimum(crypto_df['low'], crypto_df['close'].shift(1))
    tr = tr_high - tr_low
    short_bp = bp.rolling(uo_short).mean()
    short_tr = tr.rolling(uo_short).mean()
    mid_bp = bp.rolling(uo_mid).mean()
    mid_tr = tr.rolling(uo_mid).mean()
    long_bp = bp.rolling(uo_long).mean()
    long_tr = tr.rolling(uo_long).mean()

    short_avg = short_bp / short_tr
    mid_avg = mid_bp / mid_tr
    long_avg = long_bp / long_tr

    t_mid = uo_mid // uo_short
    t_long = uo_long // uo_short
    uo = 100 * ((t_long * short_avg + t_mid * mid_avg + long_avg) / (t_long + t_mid + 1))
    if toggle_indicators["uo"]:
        indicators.append(uo)
    
    return indicators

In [52]:
def get_window(df, end, period):
    return np.array(df["close"].iloc[end-period+1:end+1])

def get_labels(crypto_df, confidence):
    #y_train labels
    #either "short"(0), "hold"(1), "long"(2)
    y_train = []

    past_period = 24
    future_period = 24 
    future_dis = 24
    
    short_period = 12
    long_period = 12
    profit_margin = 0.02
    safety_margin = 0.01

    past = crypto_df["close"].rolling(window=past_period)
    future = crypto_df["close"].rolling(window=future_period)

    for i in range(len(crypto_df)):
        if i-past_period<0 or i+future_dis>=len(crypto_df):
            y_train.append(1)
        else:
            cur_price = crypto_df.iloc[i, 0]
            cur_past = get_window(crypto_df, i, past_period)
            cur_future = get_window(crypto_df, i+future_dis, future_period)
            short_past = cur_past[:short_period+1]
            short_future = cur_future[:short_period+1]

            #Mann Whitney U-test
            u_stat, p_value_u = mannwhitneyu(cur_past, cur_future)

            # Direction (using old normal-distrib test val)
            z_score = np.mean(cur_future) - np.mean(cur_past)

            #percentile constant
            check_short = np.percentile(cur_future[-long_period:], 90) <= (1-safety_margin) * np.percentile(cur_past, 10)
            check_long = np.percentile(cur_future[-long_period:], 10) >= (1+profit_margin) * np.percentile(cur_past, 90)

            #shor-term percentile constant (prevent early/late labels on trend because riskier)
            check_short2 = (np.mean(short_future) / np.mean(short_past)) <= 0.995
            check_long2 = (np.mean(short_future) / np.mean(short_past)) >= 1.005

            #significance testing
            if p_value_u <= confidence:
                if z_score < 0 and check_short and check_short2:
                    y_train.append(0)
                elif z_score > 0 and check_long and check_long2:
                    y_train.append(2)
                else:
                    y_train.append(1)
            else:
                y_train.append(1)
    return y_train

In [78]:
def get_labels(crypto_df, confidence):
    #y_train labels
    #either "short"(0), "hold"(1), "long"(2)
    y_train = []

    past_period = 30
    future_period = 30 
    future_dis = 40

    past = crypto_df["close"].rolling(window=past_period)
    future = crypto_df["close"].rolling(window=future_period)

    for i in range(len(crypto_df)):
        if i-past_period<0 or i+future_dis>=len(crypto_df):
            y_train.append(1)
        else:
            #Mann Whitney U-test
            cur_past = get_window(crypto_df, i, past_period)
            cur_future = get_window(crypto_df, i+future_dis, future_period)
            short_past = cur_past[:13]
            short_future = cur_future[:13]
            u_stat, p_value_u = mannwhitneyu(cur_past, cur_future)

            # Normal distrib hypothesis test
            z_score = (np.mean(cur_future) - np.mean(cur_past)) / max(np.std(cur_past), np.std(cur_future))
            p_value_z = 2 * (1 - stats.norm.cdf(abs(z_score)))

            #percentile constant
            percent_check_short = np.percentile(cur_future, 90) <= np.percentile(cur_past, 10)
            percent_check_long = np.percentile(cur_future, 10) >= np.percentile(cur_past, 90)

            #short block
            check_short2 = (np.mean(short_future) / np.mean(short_past)) <= 0.995
            check_long2 = (np.mean(short_future) / np.mean(short_past)) >= 1.005

            #significance testing
            if p_value_u <= confidence and p_value_z <= confidence:
                if z_score < 0 and percent_check_short and check_short2:
                    y_train.append(0)
                elif z_score > 0 and percent_check_long and check_long2:
                    y_train.append(2)
                else:
                    y_train.append(1)
            else:
                y_train.append(1)
    return y_train

In [73]:
def get_data(crypto_df, toggle_indicators, conf):
    indicators = get_indicators(crypto_df, toggle_indicators)
    y_train = get_labels(crypto_df, conf)
    
    #Clearing out bad beginning + end data
    left = 200
    right = 200
    indicator_period = 5
    period_indicators = []
    for i in range(len(indicators)):
        for j in range(indicator_period):
            shifted = np.roll(indicators[i], j)
            period_indicators.append(shifted[left:-right])

    X_train = np.transpose(np.array(period_indicators))
    y_train = y_train[left:-right]

    return X_train, y_train

def create_model(crypto_df, toggle_indicators, conf, crypto_name: str):
    X_train, y_train = get_data(crypto_df, toggle_indicators, conf)
    rand_state = 20
    
    #Creating ML models for decision-making
    # Random Forest
    #random_forest = RandomForestClassifier(criterion="entropy", n_estimators=10, max_depth=5, random_state=20)
    cnt0 = np.sum(np.array(y_train) == 0)
    cnt1 = np.sum(np.array(y_train) == 1)
    cnt2 = np.sum(np.array(y_train) == 2)
    random_forest = RandomForestClassifier(
        criterion="entropy", #gini
        n_estimators=500, 
        max_depth=10,
        class_weight={0: 1, 1: 1, 2: 1},
        random_state=rand_state
    )
    
    # XGB Boost
    xgb_model = XGBClassifier(
        n_estimators=1000,
        max_depth=6,
        learning_rate=0.01,
        booster="gblinear", #dart, gbtree
        # rate_drop=0.05,
        # skip_drop=0.1,
        objective="multi:softmax", #multi:softprob
        num_class=3,
        random_state=rand_state
    )
    # Support Vector Machine
    svm_model = SVC(
        kernel="sigmoid", 
        C=1.0,
        gamma="scale",
        decision_function_shape="ovo",
        max_iter=1000,
        tol=0.01,
        random_state=rand_state
    )
    # Multi-class logistic regression
    log_reg = LogisticRegression(
        multi_class="ovr", #multinomial bad
        solver="newton-cg", #lbfgs 
        #penalty="elasticnet",
        #l1_ratio=0.2,
        C=1.0,
        max_iter=25, #25-30 seems good
        random_state=rand_state
    )
    # Stacking
    base_models = [ 
        ("random_forest", random_forest), 
        ("xgb", xgb_model), 
        #("svc", svm_model),
        ("logistic_regression", log_reg) 
    ]

    meta_model = LogisticRegression(multi_class="ovr", solver="newton-cg", max_iter=25, random_state=rand_state)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_state)
    stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=cv)


    random_forest.fit(X_train, y_train)
    model = random_forest

    #Exporting model
    model_key = crypto_name
    file_name = qb.object_store.get_file_path(model_key)
    joblib.dump(model, file_name)    
    return model

In [54]:
def visualize(crypto_df, y_pred):
    crypto_df = crypto_df.iloc[200:-200]
    
    x_vals = np.arange(1, len(crypto_df)-400 + 1)
    y_vals = crypto_df.iloc[200:-200, 0]
    y_pred = y_pred[200:-200]

    colors = []
    for i in range(len(y_pred)):
        if y_pred[i] == 0:
            colors.append("red")
        elif y_pred[i] == 1:
            colors.append("gray")
        else:
            colors.append("green")

    print(x_vals.shape)
    print(y_vals.shape)

    plt.figure(figsize=(250, 70))

    plt.scatter(x_vals, y_vals, c=colors)
    plt.xlabel("prices")
    plt.ylabel("times (hr)")
    plt.show()

In [79]:
qb = QuantBook()
start_time = datetime(2017, 1, 1)  # no data on btcusdt/binance until 8/11/2018
end_time = datetime(2022, 1, 1) #2022, 1, 1 for BTCUSDT
end_time_test = datetime(2022, 11, 1) #2024, 11, 1 for full test

#Indicators
toggle_indicators = {
    "macd": True,
    "vwap": True,
    "rsi": True,
    "adx": True,
    "cci": True,
    "stoch": True,
    "hma": False, #buggy indicator
    "cmo": True,
    "uo": True
}

#Cryptos: https://www.quantconnect.com/docs/v2/writing-algorithms/datasets/coinapi
# cryptos = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "LINKUSDT", "AVAXUSDT"]
# conf = [0.95, 0.95, 0.95, 0.95, 0.95, 0.95]
cryptos = ["BTCUSDT"]
confs = [0.95]
model = 1

#for i in range(len(cryptos)):
crypto = cryptos[0]
conf = confs[0]

#market=Market.BINANCE
crypto_symbol = qb.add_crypto(crypto, Resolution.HOUR, market=Market.BINANCE).symbol
crypto_df = qb.history(TradeBar, crypto_symbol, start_time, end_time)
model = create_model(crypto_df, toggle_indicators, conf, crypto)

df_test = qb.history(TradeBar, crypto_symbol, end_time, end_time_test)
X_test, y_test = get_data(df_test, toggle_indicators, conf)
y_pred = model.predict(X_test)

In [80]:
df_test = qb.history(TradeBar, crypto_symbol, end_time, end_time_test)
X_test, y_test = get_data(df_test, toggle_indicators, conf)
y_pred = model.predict(X_test)

In [81]:
visualize(df_test, y_test)
visualize(df_test, y_pred)

In [82]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Model momentum accuracy: {accuracy}")
cm = confusion_matrix(y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues) 
plt.title("Confusion Matrix") 
plt.show()

In [59]:
# crypto_symbol = qb.add_crypto("BTCUSDT", Resolution.HOUR, market=Market.BINANCE).symbol
# df_test = qb.history(TradeBar, crypto_symbol, end_time, end_time_test)
# X_test, y_test = get_data(df_test, toggle_indicators, conf)
# y_pred = model.predict(X_test)
# accuracy = accuracy_score(y_test, y_pred)
# print(f"Model momentum accuracy: {accuracy}")
# cm = confusion_matrix(y_test, y_pred)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot(cmap=plt.cm.Blues) 
# plt.title("Confusion Matrix") 
# plt.show()

In [60]:
# import pandas as pd 
# import numpy as np
# import math
# from datetime import datetime
# from sklearn.ensemble import RandomForestClassifier
# import joblib

# from scipy.stats import mannwhitneyu
# import scipy.stats as stats
# import xgboost as xgb
# from xgboost import XGBClassifier
# from sklearn.svm import SVC
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import StackingClassifier
# from sklearn.model_selection import StratifiedKFold
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
# from sklearn.utils.class_weight import compute_class_weight

# def get_indicators(crypto_df, toggle_indicators):
#     indicators = []

#     #constants
#     macd_short = 12
#     macd_long = 24
#     macd_signal = 8
#     vwap_period = 24
#     rsi_period = 24
#     adx_period = 24
#     cci_period = 24*3

#     #MACD
#     ema_short = crypto_df["close"].ewm(span=macd_short, adjust=False).mean()
#     ema_long = crypto_df["close"].ewm(span=macd_long, adjust=False).mean()
#     ema_signal = crypto_df["close"].ewm(span=macd_signal, adjust=False).mean()

#     macd = ((ema_short - ema_long) - ema_signal) 
#     if toggle_indicators["macd"]:
#         indicators.append(macd)

#     #vwap
#     avg_price = (crypto_df["close"] + crypto_df["high"] + crypto_df["low"]) / 3
#     weighted_price = (avg_price * crypto_df["volume"]).rolling(window=vwap_period).sum()
#     volume = crypto_df["volume"].rolling(window=vwap_period).sum()
#     vwap = weighted_price / volume
#     #case when volume = 0 -> use prev vwap
#     for i in range(len(vwap)):
#         if np.isnan(vwap.iloc[i]) and i > 0:
#             vwap.iloc[i] = vwap.iloc[i-1]
#     vwap = (crypto_df["close"] - vwap) 
#     if toggle_indicators["vwap"]:
#         indicators.append(vwap)

#     #rsi
#     diffs = crypto_df["close"].diff()
#     gain = (diffs.where(diffs > 0, 0)).rolling(window=rsi_period).mean()
#     loss = (diffs.where(diffs < 0, 0)).rolling(window=rsi_period).mean()
#     rs = gain / np.abs(loss)
#     rsi = 100 - 100 / (1 + rs)
#     #case when loss = 0 -> max momentum
#     rsi = rsi.apply(lambda x: 100 if np.isnan(x) else x)
#     if toggle_indicators["rsi"]:
#         indicators.append(rsi)

#     #adx
#     high_low = crypto_df["high"] - crypto_df["low"]
#     high_close = np.abs(crypto_df["high"] - crypto_df["close"].shift())
#     low_close = np.abs(crypto_df["low"] - crypto_df["close"].shift())
#     TR = high_low.combine(high_close, max).combine(low_close, max)

#     plus_dm = crypto_df["high"].diff()
#     minus_dm = crypto_df["low"].diff()
#     plus_dm[plus_dm < 0] = 0
#     minus_dm[minus_dm > 0] = 0

#     smooth_TR = TR.rolling(window=adx_period).mean()
#     smooth_plus_dm = plus_dm.rolling(window=adx_period).mean()
#     smooth_minus_dm = abs(minus_dm.rolling(window=adx_period).mean())
#     #Limiting denom
#     smooth_TR = smooth_TR.apply(lambda x: 0.01 if abs(x) < 0.01 else x)

#     plus_DI = (smooth_plus_dm / smooth_TR) * 100
#     minus_DI = (smooth_minus_dm / smooth_TR) * 100
#     adx_denom = plus_DI + minus_DI
#     dx = ((plus_DI - minus_DI) / adx_denom) * 100
#     #case when +DI + -DI = 0 -> no price movement
#     dx = dx.apply(lambda x: 0 if np.isnan(x) else x)
#     adx = dx.rolling(window=adx_period).mean()
#     if toggle_indicators["adx"]:
#         indicators.append(adx)

#     #cci
#     tp = (crypto_df["high"] + crypto_df["low"] + crypto_df["close"]) / 3

#     sma_tp = tp.rolling(window=cci_period).mean()
#     md = tp.rolling(window=cci_period).apply(lambda x: np.fabs(x - x.mean()).mean())

#     # 0.015 is apparently a standard constant for 20-30% of cci values to indicate trend
#     cci = (tp - sma_tp) / (0.015 * md)
#     if toggle_indicators["cci"]:
#         indicators.append(cci)
    
#     return indicators


# def get_window(df, end, period):
#     return np.array(df["close"].iloc[end-period+1:end+1])

# def get_labels(crypto_df, confidence):
#     #y_train labels
#     #either "short"(0), "hold"(1), "long"(2)
#     y_train = []

#     past_period = 30
#     future_period = 30 
#     future_dis = 30

#     past = crypto_df["close"].rolling(window=past_period)
#     future = crypto_df["close"].rolling(window=future_period)

#     for i in range(len(crypto_df)):
#         if i-past_period<0 or i+future_dis>=len(crypto_df):
#             y_train.append(1)
#         else:
#             #Mann Whitney U-test
#             cur_past = get_window(crypto_df, i, past_period)
#             cur_future = get_window(crypto_df, i+future_dis, future_period)
#             u_stat, p_value_u = mannwhitneyu(cur_past, cur_future)

#             # Normal distrib hypothesis test
#             z_score = (np.mean(cur_future) - np.mean(cur_past)) / max(np.std(cur_past), np.std(cur_future))
#             p_value_z = 2 * (1 - stats.norm.cdf(abs(z_score)))

#             #percentile constant
#             percent_check_short = np.percentile(cur_future, 90) <= np.percentile(cur_past, 10)
#             percent_check_long = np.percentile(cur_future, 10) >= np.percentile(cur_past, 90)

#             #significance testing
#             if p_value_u <= confidence and p_value_z <= confidence:
#                 if z_score < 0 and percent_check_short:
#                     y_train.append(0)
#                 elif z_score > 0 and percent_check_long:
#                     y_train.append(2)
#                 else:
#                     y_train.append(1)
#             else:
#                 y_train.append(1)
#     return y_train

# def get_data(crypto_df, toggle_indicators, conf):
#     indicators = get_indicators(crypto_df, toggle_indicators)
#     y_train = get_labels(crypto_df, conf)
    
#     #Clearing out bad beginning + end data
#     left = 200
#     right = 200
#     indicator_period = 5
#     period_indicators = []
#     for i in range(len(indicators)):
#         for j in range(indicator_period):
#             shifted = np.roll(indicators[i], j)
#             period_indicators.append(shifted[left:-right])
    
#     # for i in range(len(indicators)):
#     #     for j in range(len(indicators[i])):
#     #         if np.isnan(indicators[i][j]):
#     #             print(f"NAN!!: {i}, {j}, len: {len(indicators[i])}")

#     X_train = np.transpose(np.array(period_indicators))
#     y_train = y_train[left:-right]

#     return X_train, y_train

# # class Custom_SVM:
# #     def __init__(self, kernel_, C_, max_iter_, random_state_):
# #         self.short_svm = SVC(kernel=kernel_, C=C_, max_iter=max_iter_, probability=True, random_state=random_state_)
# #         self.long_svm = SVC(kernel=kernel_, C=C_, max_iter=max_iter_, probability=True, random_state=random_state_)
# #         self.stay_svm = SVC(kernel=kernel_, C=C_, max_iter=max_iter_, probability=True, random_state=random_state_)
    
# #     def fit(self, X, y):
# #         y_short = np.copy(y)
# #         y_short[y_short == 2] = 1
# #         y_long = np.copy(y)
# #         y_long[y_long == 0] = 1
# #         self.short_svm.fit(X, y_short)
# #         self.long_svm.fit(X, y_long)
# #         self.stay_svm.fit(X, y)
    
# #     def predict(self, X):
# #         # Get predictions from both models
# #         short = self.short_svm.predict(X)
# #         long = self.long_svm.predict(X)
# #         stay = self.stay_svm.predict(X)
        
# #         # Apply custom condition: output 1 if both models predict 1, else 0
# #         custom_preds = []
# #         for i in range(len(short)):
# #             if short[i] == 0 and long[i] == 1 and stay[i] == 0:
# #                 custom_preds.append(0)
# #             elif short[i] == 1 and long[i] == 2 and stay[i] == 2:
# #                 custom_preds.append(2)
# #             else:
# #                 custom_preds.append(1)
# #         return custom_preds

# def create_model(crypto_df, toggle_indicators, conf, crypto_name: str):
#     X_train, y_train = get_data(crypto_df, toggle_indicators, conf)
#     rand_state = 20

#     #Loss Matrix implementations:
#     loss_matrix = np.array([
#         [0, 3, 100],
#         [1, 0, 1],
#         [100, 3, 0]
#     ])

#     #Creating ML models for decision-making
#     # Random Forest
#     #random_forest = RandomForestClassifier(criterion="entropy", n_estimators=10, max_depth=5, random_state=20)
#     random_forest = RandomForestClassifier(
#         criterion="entropy", #gini
#         n_estimators=10, 
#         max_depth=5,
#         random_state=rand_state
#     )
    
#     # XGB Boost
#     xgb_model = XGBClassifier(
#         n_estimators=200,
#         max_depth=6,
#         learning_rate=0.01,
#         booster="gblinear", #dart, gbtree
#         # rate_drop=0.05,
#         # skip_drop=0.1,
#         objective="multi:softmax", #multi:softprob
#         num_class=3,
#         random_state=rand_state
#     )
#     # Support Vector Machine
#     svm_model = SVC(
#         kernel="sigmoid", 
#         C=1.0,
#         gamma="scale",
#         decision_function_shape="ovo",
#         max_iter=1000,
#         tol=0.01,
#         random_state=rand_state
#     )
#     # Multi-class logistic regression
#     log_reg = LogisticRegression(
#         multi_class="ovr", #multinomial bad
#         solver="newton-cg", #lbfgs 
#         #penalty="elasticnet",
#         #l1_ratio=0.2,
#         C=1.0,
#         max_iter=25, #25-30 seems good
#         random_state=rand_state
#     )
#     # Stacking
#     base_models = [ 
#         ("random_forest", random_forest), 
#         ("xgb", xgb_model), 
#         #("svc", svm_model),
#         ("logistic_regression", log_reg) 
#     ]

#     meta_model = LogisticRegression(multi_class="ovr", solver="newton-cg", max_iter=25, random_state=rand_state)
#     cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=rand_state)
#     stack = StackingClassifier(estimators=base_models, final_estimator=meta_model, cv=cv)


#     random_forest.fit(X_train, y_train)
#     model = random_forest

#     #Exporting model
#     model_key = crypto_name
#     file_name = qb.object_store.get_file_path(model_key)
#     joblib.dump(model, file_name)    
#     return model

# qb = QuantBook()
# start_time = datetime(2017, 1, 1)  # no data on btcusdt/binance until 8/11/2018
# end_time = datetime(2023, 1, 1) #2022, 1, 1 for BTCUSDT
# end_time_test = datetime(2024, 11, 1)

# #Indicators
# toggle_indicators = {
#     "macd": True,
#     "vwap": True,
#     "rsi": True,
#     "adx": True,
#     "cci": True
# }

# #Cryptos: https://www.quantconnect.com/docs/v2/writing-algorithms/datasets/coinapi
# # cryptos = ["BTCUSDT", "ETHUSDT", "BNBUSDT", "SOLUSDT", "LINKUSDT", "AVAXUSDT"]
# # conf = [0.95, 0.95, 0.95, 0.95, 0.95, 0.95]
# cryptos = ["BTCUSDT"]
# confs = [0.95]
# model = 1
# for i in range(len(cryptos)):
#     crypto = cryptos[i]
#     conf = confs[i]

#     #market=Market.BINANCE
#     crypto_symbol = qb.add_crypto(crypto, Resolution.HOUR, market=Market.BINANCE).symbol
#     crypto_df = qb.history(TradeBar, crypto_symbol, start_time, end_time)
#     model = create_model(crypto_df, toggle_indicators, conf, crypto)

#     df_test = qb.history(TradeBar, crypto_symbol, end_time, end_time_test)
#     X_test, y_test = get_data(df_test, toggle_indicators, conf)
#     y_pred = model.predict(X_test)